In [1]:
!pip install galois
!pip install scipy

In [2]:
import galois
import numpy as np
import time
import os
from scipy import stats

In [3]:
#definisco un campo di Galois come nell'esempio 
field = galois.GF(2**3)

In [4]:
#definisco una matrice k x (n-k)
k = 3 #le righe
n_k = 4 #le colonne

#N.B: Less usa q = dimensione campo: 127; k = 126; n = 252 visto da pdf myLESS_CF pagina 38 di 45

In [52]:
A = field(np.random.randint(0, field.order, size=(k, n_k)))
print(A)

[[4 3 5 3]
 [2 7 2 5]
 [4 1 5 1]]


In [53]:
matrice_bin1 = np.array([
    [0, 0, 1],
    [1, 0, 0],
    [0, 1, 0]
])

matrice_bin1 = field(matrice_bin1)

matrice_bin2 = np.array([
    [1, 0, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [0, 1, 0, 0]
])

matrice_bin2 = field(matrice_bin2)

In [54]:
A_primo = field(matrice_bin1 @ A @ matrice_bin2)
print(A_primo)

[[4 1 1 5]
 [4 3 3 5]
 [2 5 7 2]]


In [55]:
def duplicati(etichetta):
    #se torna true --> lunghezze diverse --> ci sono duplicati
    return len(etichetta) != len(set(etichetta))

#fa il procedimento di ordinamento tra le righe (che pero sono già ordinate al loro interno)
def ordina_matrice(etichetta):
    print("Entro in ordina matrice")

    #voglio vedere durata esecuzione con meno colonne
    start = time.perf_counter()
    if not duplicati(etichetta):
        indici = np.argsort(etichetta)
        end = time.perf_counter()
        #finisce confronto con colonne ridotte
        tempo = (end-start)*1000
        return True, indici, tempo
    end = time.perf_counter()
    #finisce confronto con colonne ridotte
    tempo = (end-start)*1000
    return False, [], tempo
    
# Funzione per ordinare le righe con contatori di swap. In num_confronti metto confronti in # sufficienti metto confronti1. 
def ordina_per_riga_con_contatori(matrice, num_swap_righe):
    #tengo posizioni originali per confrontarle con quelle cambiate e quindi vedere quanti swap di riga ho fatto
    print("Funzione ordina_per_riga")
    posizioni_originali = np.arange(matrice.shape[0])
    
    #se non metto l'ordinamento della riga non ridà
    ord = []
    for riga in matrice:
        ord.append(np.sort(riga))
    ord = np.array(ord)
    print(ord)

    etichetta = np.array([hash(tuple(riga.tolist())) for riga in ord])    
    print(etichetta)
 
    ris_confronto, indici_ordinamento, tempo = ordina_matrice(etichetta)
    print(matrice[indici_ordinamento])
    #print(etichetta[indici_ordinamento])
    #se torno true quindi due matrici sono uguali
    if ris_confronto:
        # Conta gli swap solo quando la posizione delle righe cambia
        for i, indice in enumerate(indici_ordinamento):
            if indice != posizioni_originali[i]:
                num_swap_righe += 1
        return matrice[indici_ordinamento], num_swap_righe, tempo
    else:
        return [], num_swap_righe, tempo


#POTREI DOVER IMPLEMENTARE UNA COSA SIMILE A SOPRA PER IL COUNT DEI CONFRONTI (LEXSORT NON CE L'HA)
#ordinare le colonne con contatori di ordinamenti e swap
def ordina_per_colonna_con_contatori(matrice, righe, num_swap_colonne):
    print("Funzione ordina per colonna")
    #tengo posizioni originali per confrontarle con quelle cambiate e quindi vedere quanti swap di riga ho fatto
    posizioni_originali = np.arange(matrice.shape[1])
    
    # Trova gli indici di ordinamento usando np.lexsort sulle righe, se c'è conflitto passa a el seconda riga
    indici_ordinamento = np.lexsort([matrice[rig, :] for rig in righe[::-1]])
        
    # Conta gli swap solo quando la posizione delle colonne cambia
    for i, indice in enumerate(indici_ordinamento):
        if indice != posizioni_originali[i]:
            num_swap_colonne += 1
    
    #riordina la matrice in base agli indici trovati
    matrice_ordinata = matrice[:, indici_ordinamento]
    return matrice_ordinata, num_swap_colonne

#qui faccio il test
def test_algoritmo(matrice1, matrice2):
    righe = range(k)
    print("Matrice1: ", matrice1)
    print("Matrice2: ", matrice2)


    ordinata1, num_swap_righe, tempo = ordina_per_riga_con_contatori(matrice1, num_swap_righe=0)
    if len(ordinata1) == 0:
        return False
    ordinata_totale_1, num_swap_colonne = ordina_per_colonna_con_contatori(ordinata1, righe, num_swap_colonne=0)
    print("\nOrdinata totale 1:", ordinata_totale_1)
    
    ordinata2, num_swap_righe, tempo = ordina_per_riga_con_contatori(matrice2, num_swap_righe=0)
    if len(ordinata2) == 0:
        return False
    ordinata_totale_2, num_swap_colonne = ordina_per_colonna_con_contatori(ordinata2, righe, num_swap_colonne=0)    
    print("\nOrdinata totale 2:", ordinata_totale_2)

    print(np.array_equal(ordinata_totale_1, ordinata_totale_2))
    if np.array_equal(ordinata_totale_1, ordinata_totale_2):
        print("Le due matrici hanno la stessa CF.")
        return True
    else:
        print("Le due matrici hanno CF diversa")
        return False   


In [56]:
test_algoritmo(A, A_primo)

Matrice1:  [[4 3 5 3]
 [2 7 2 5]
 [4 1 5 1]]
Matrice2:  [[4 1 1 5]
 [4 3 3 5]
 [2 5 7 2]]
Funzione ordina_per_riga
[[3 3 4 5]
 [2 2 5 7]
 [1 1 4 5]]
[  185617158705320726 -7940537775921757111 -3752905274096053005]
Entro in ordina matrice
[[2 7 2 5]
 [4 1 5 1]
 [4 3 5 3]]
Funzione ordina per colonna

Ordinata totale 1: [[2 2 5 7]
 [4 5 1 1]
 [4 5 3 3]]
Funzione ordina_per_riga
[[1 1 4 5]
 [3 3 4 5]
 [2 2 5 7]]
[-3752905274096053005   185617158705320726 -7940537775921757111]
Entro in ordina matrice
[[2 5 7 2]
 [4 1 1 5]
 [4 3 3 5]]
Funzione ordina per colonna

Ordinata totale 2: [[2 2 5 7]
 [4 5 1 1]
 [4 5 3 3]]
True
Le due matrici hanno la stessa CF.


True